<a href="https://colab.research.google.com/github/SalmanRafiullah/garbage-classification/blob/master/InceptionV3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import necessary libraries
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pickle
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from google.colab import drive
import os

#Install tf-nightly
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass

import tensorflow as tf

Using TensorFlow backend.


     |████████████████████████████████| 533.2MB 28kB/s 
     |████████████████████████████████| 460kB 56.1MB/s 
     |████████████████████████████████| 2.9MB 39.8MB/s 
     |████████████████████████████████| 2.8MB 34.3MB/s 
     |████████████████████████████████| 778kB 46.8MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2


In [2]:
#Mount Google Drive
drive.mount('/content/drive')

#Assign directory path for test, train and validation data
test_data_dir = '/content/drive/My Drive/split-garbage-dataset/split-garbage-dataset/test'
train_data_dir = '/content/drive/My Drive/split-garbage-dataset/split-garbage-dataset/train'
valid_data_dir = '/content/drive/My Drive/split-garbage-dataset/split-garbage-dataset/valid'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Load train images from directory
train_generator = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1. / 255,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=40,
    ).flow_from_directory(
        directory=train_data_dir,
        target_size=(300, 300),
        class_mode = 'categorical',
    )

#Load validation images from directory
valid_generator = ImageDataGenerator(
        rescale=1 / 255,
    ).flow_from_directory(
        directory=valid_data_dir,
        target_size=(300, 300),
        class_mode = 'categorical',
    )

#Load test images from test directory
test_generator = ImageDataGenerator(
        rescale=1 / 255,
    ).flow_from_directory(
        directory=test_data_dir,
        target_size=(300, 300),
        class_mode = 'categorical',
    )

Found 1768 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


In [4]:
#len(train_generator), len(valid_generator), len(test_generator),

(56, 11, 14)

In [0]:
#Pending code for visualizing sample photos

In [5]:
train_generator[0][0].shape

(32, 300, 300, 3)

In [6]:
#Define model to train the data on
base_model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3))
#Loading pretrained weights as Trasnsfer Learning fro Inception V3
#https://www.kaggle.com/careyai/inceptionv3-full-pretrained-model-instructions
base_model.load_weights('/content/drive/My Drive/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False
#Defining layers for model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    #Conv2D(1024,3, strides = 2, padding="same", activation='relu'),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    #Flatten(),
    Dense(6, activation='softmax')
])














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#Define optimizer and learning rate
opt = optimizers.nadam(lr=0.001)
#Compile the model
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
#Print model summary
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 6150      
Total params: 23,907,110
Trainable params: 2,104,326
Non-trainable params: 21,802,784
________________________________

In [0]:


batch_size = 200
epochs = 300
patience = 10

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

filepath = "/content/drive/My Drive/model/model_{epoch:02d}-{val_acc:.2f}.h5"
#filepath = "model_{epoch:02d}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
checkpoint2 = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

rlrp = ReduceLROnPlateau(monitor='acc', factor=0.1, patience=patience, min_delta=1E-7)

callbacks_list = [rlrp,checkpoint1,checkpoint2]

In [9]:
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/300
8/8 [==============================] - 96s 12s/step - loss: 2.5951 - acc: 0.2266 - val_loss: 1.9408 - val_acc: 0.3125

Epoch 00001: saving model to /content/drive/My Drive/model/model_01-0.31.h5

Epoch 00001: acc improved from -inf to 0.22656, saving model to /content/drive/My Drive/model/model_01-0.31.h5
Epoch 2/300
8/8 [==============================] - 86s 11s/step - loss: 1.7202 - acc: 0.3164 - val_loss: 1.2897 - val_acc: 0.5000

Epoch 00002: saving model to /content/drive/My Drive/model/model_02-0.50.h5

Epoch 00002: acc improved from 0.22656 to 0.31641, saving model to /content/drive/My Drive/model/model_02-0.50.h5
Epoch 3/300
8/8 [==============================] - 86s 11s/step - loss: 1.3449 - acc: 0.4531 - val_loss: 1.2858 - val_acc: 0.5312

Epoch 00003: saving model to /content/drive/My Drive/model/model_03-0.53.h5

Epoch 00003: acc improved from 0.31641 to 0.45312, savi

In [0]:
from google.colab import files
import pandas as pd

hist_df = pd.DataFrame(history.history) 
hist_df.to_csv('trainHistoryDict.csv')
files.download('trainHistoryDict.csv')